In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import os
import numpy as np

## Install YOLOv8

In [ ]:
# !pip install ultralytics==8.3.19
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.5/112.6 GB disk)


## Install Roboflow Supervision

In [58]:
!pip install supervision[assets]==0.24.0

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.24.0


## Load pre-trained YOLOv8 model

In [59]:
from ultralytics import YOLO

model = YOLO(
    os.path.join("/content/drive/MyDrive/Papaya-counter/results/complete_train/yolo11m", 'train/weights/best.pt')
)

## Predict and annotate whole video

In [64]:
# settings
LINE_START = sv.Point(0, 480/2 - 20)
LINE_END = sv.Point(640, 480/2 - 20)

SELECTED_CLASS_IDS = [0]
SOURCE_VIDEO_PATH = f"/content/drive/MyDrive/Papaya-counter/Teste_Video_resized.mp4"
TARGET_VIDEO_PATH = f"/content/drive/MyDrive/Papaya-counter/results/complete_train/yolo11m/FINAL.mp4"

In [65]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=640, height=480, fps=30, total_frames=1231)

In [66]:
import cv2
# counter
# Define the text, font, and position
counter_font = cv2.FONT_HERSHEY_SIMPLEX
counter_font_scale = 2
counter_text_color = (0, 255, 0)
counter_text_black = (0, 0, 0)
counter_thickness = 4
counter_position = (10, 50)

In [67]:
# create BYTETracker instance
byte_tracker = sv.ByteTrack(
    track_activation_threshold=0.2,
    lost_track_buffer=30,
    minimum_matching_threshold=0.9,
    frame_rate=30,
    minimum_consecutive_frames=2)

byte_tracker.reset()

# create LineZone instance, it is previously called LineCounter class
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator, LabelAnnotator, and TraceAnnotator
box_annotator = sv.BoxAnnotator(thickness=1)
label_annotator = sv.LabelAnnotator(text_thickness=1, text_scale=0.5, text_color=sv.Color.BLACK, text_padding=0)

# define call back function to be used in video processing
def callback(frame: np.ndarray, index: int) -> np.ndarray:

    # model prediction on single frame and conversion to supervision Detections
    # Conf = 0.25 due to the necessity of a high sensibility/recall
    results = model(frame, conf=0.10, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, SELECTED_CLASS_IDS)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    five_seconds = 30*5
    thirty_five_seconds = 30*35
    if (index >= five_seconds) and (index <= thirty_five_seconds):
        # update line counter
        line_zone.trigger(detections)

    annotated_frame = frame.copy()
    annotated_frame = cv2.putText(annotated_frame, str(line_zone.in_count), counter_position, counter_font, counter_font_scale, counter_text_black, counter_thickness+1, cv2.LINE_AA)
    annotated_frame = cv2.putText(annotated_frame, str(line_zone.in_count), counter_position, counter_font, counter_font_scale, counter_text_color, counter_thickness, cv2.LINE_AA)
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    # return frame with box and line annotated result

    return annotated_frame

# process the whole video
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)